#  Routing with IPv4
<i>Adapted for use with FABRIC from [IPv4 Routing Assignment](https://groups.geni.net/geni/wiki/GENIEducation/SampleAssignments/IPRouting/Procedure)</i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

This is the second step in this assignment. To go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb) 
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/IPV4%20routing/figures/IPV4Top.png?raw=true">
<br>The goal of this exercise is to setup the routing as indicated in the figure above; i.e. packets from A sent to IP address 192.168.2.12 on node C should be routed via node B. In order to create this routing behavior you will need to modify the routing tables in your nodes using the [linux route command](https://www.hscripts.com/tutorials/linux-commands/route.html)

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = 'IPv4_Routing'
try:
    slice = fablib.get_slice(slice_name)   
    slice.list_nodes()
except Exception as e:
    print(f"Get Slices Fail: {e}")

## 2. Guided Experiment
The goal of this assignment is to understand what the effects of routing commands are and understand what the routing table represents. In this assignment you will set up the routing in the nodes as it was indicated in the figure below. Packets from A sent to IP address 192.168.2.12 on node C should be routed via node B. In order to create this routing behavior you will need to modify the routing tables in your nodes using the [linux route command](https://www.hscripts.com/tutorials/linux-commands/route.html)

#### The Route Command Introduction
The Linux route command follows the following general structure: `route {operation} {Destination} {Gateway} {InterfaceGateway}` knowing this, if you run `route` or `route -n` you will be able to display the current routing table. The routing table will display all of the available connections "routes'' that this host is connected to and their attributes. The following are the descriptions of what each of the columns represent:
	- **Destination** This is the IP address of the host's connection, you can use this IP to connect with the other host
	- **Gateway** This is the 'Hop' IP, if a host would like to communicate with another host, but does not have a direct connection, you can add the IP of a host who shares both of the nodes as a connection and route through this intermediary host to form a connection between yourself and the previously unreachable host
	- **Genmask** If you decide to add a network you will have to add a subnet mask that represents the available ips this connection can form, for example 11.0.0.0/16 would get attributed the net mask of 255.255.0.0 because you would be able to ping from 11.0.0.0 all the way to the ip address 11.0.255.255.
	- **Flags** There exist 3 flags when using the route command that represent the current status of the route: "U" means that the route is up and available, "H" means that the target is a host, "G" means that the target is a gateway.
	- **Iface** This column states what device the route is connected to.

The route command can take some arguments to add and delete routes; the following section will walk you through how to use each parameter.
1. Operation *Required*
	- **add** This flag asks the route command to *create* a new route. Keep in mind you can not modify an existing route, if you want to modify you will need to delete it first and then create a new one.
	- **del** This flag asks the route command to *delete* an existing route
2. Destination *Required*
	- **{IP}** If you only provide an IP the route command will treat the IP as if it contained the -host flag
	- **{Flag} {IP}** The destination section requires a flag and an IP to be completed. The IP in this section is the destination  IP of the host that you want to route to.
  	  - **-host {IP}** The host tag asks the route command to route to a single host.
  	  - **-net {IP} netmask {MASK}** The "net" tag asks the route command to create a connection to a range of possible connections. The MASK refers to a section of the ip that remains static when creating the range as described above.
3. Gateway *Optional*
	- **-gw {IP}**  If you want to route through another connection that you already have, this is where you would add the IP of the host you share in common.
4. InterfaceGateway *Required*
	- **dev {Device}** In this section you need to provide the name of the device this connection will use to route, the device must be connected to the destination host or to the node that is shared with the destination host

### 2.1 Routing Tables
First we will take a look at our routing tables, to do this simply type ```route``` once you have logged into a node. Your output should look similar to this:
<br><img src="./figures/IPv4_RouteB.png"><br>
In this example we are using Node B to see the available connections with the other nodes. Going by the image we can see some information about the connections at node B:
- node A (192.168.1.10) and node C (192.168.2.12) are both routed as networks, using the mask 255.255.255.0 meaning:
  - if we were to add other nodes to these ports (using the same ip network) we would not have to change the route.
- we can also see that nodeA is using the device "enp7s0" while node C is using the device "enp8s0"

**notice** that in these experiments devices do not share lan, so if we tried to route directly to node C using the device "enp7s0" then this connection would fail because this device can only see 192.168.1.10.

### 2.2 Routing from Node to Node
In this section we will introduce to you how to create a route from Node A to node C using the **green** route in the figure below, after we have completed this introduction your task in the assignment will be to create the **Red** route. Unlike the original routes on node B, for the connection from node A to node C, we will add direct routes.
1. Open 2 terminals and login to both Node A and Node C
2. in the node A terminal ping Node C (192.168.3.12)

you should see that the host is unreachable because we have not added a route yet, you can confirm that this is the case by using the route command to display the routing tables:
<br><img src="./figures/IPv4_RouteA.png"><br>As we can see node A only has routes to node B and so does node C

3. to create a route we will have to run ```route {operation} {Destination} {InterfaceGateway}```
	- **{Operation}** in this case we will add a route: ```add```
	- **{Destination}** in this case we want to go to node C's 192.168.3.12 and we want to add a host not a network so the command is: ```-h 192.168.3.12```
	- **{InterfaceGateway}** finally we know that the device connected to C is enp7s0: ```dev enp7s0```
	the final command should then be : ```sudo route add 192.168.3.12 dev enp7s0 ```
4. run ```sudo route add 192.168.3.12 dev enp7s0 ``` on node A, the reason why we added the keyword sudo is because we need root privileges to add routes.
5. Ping node C one more time: ```ping 192.168.3.12 -c 1```, the ping should have failed, but why?

simply you must keep in mind that routes are local to the machine, so if you try to route to another machine, by default the other machine does not know how to get back to your original location, to fix this issue we need to add the routes both ways.

6. add the route to node A from node C using a modified version of the command ```sudo route add 192.168.3.10 dev enp7s0 ``` (make sure the information matches your environment)
7. if you complete the steps successfully you have the following output:
<br><img src="./figures/IPv4_RouteA_Out.png"><br>


## 3. Assignment
Remember: the goal of this assignment is to create a route so that you an send packets from node A to IP address 192.168.2.12 on node C, this traffic should be routed via node B and is represented by the **red** route on the image bellow
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/IPV4%20routing/figures/ExerciseIPv4.png?raw=true"><br>

1. What happens when you traceroute from A to IP address 192.168.2.12 before you setup the static routes? Why?
	(Include the output of the traceroute in your writeup)
2. Setup the routing from A to 192.68.2.12 so that it goes through B. Was it enough to just modify the routing tables? What else did you need to change in order for the traffic to flow? Ensure that you have connectivity by running a ping from A to 192.168.2.12
	(In your writeup include all the commands you ran and a screenshot of the routing configuration)
3. What happens when you traceroute from A to IP address 192.168.2.12 after you setup the static routes?
	(In your writeup include a screenshot of the traceroute output.)

## Cleanup Resources

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")